# Homework 6 - Group 30
## Team Member:
### Tianhao Chen (Henry) - NetID: tc30 
### Yizhan Xue - NetID: yizhanx2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bqplot
import ipywidgets

#### Read the csv file, load the dataset

In [2]:
dataset_url = 'https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_bcubcg_fall2022/main/data/licenses_fall2022.csv'
licenses = pd.read_csv(dataset_url)
licenses.head()

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason
0,1189509,DETECTIVE BOARD,PERMANENT EMPLOYEE REGISTRATION,129446286,NOT RENEWED,N,NaN,EILEEN,NaN,SANTACRUZ,...,None,None,None,N,03/18/2022,None,None,None,None,None
1,801037,DETECTIVE BOARD,FIREARM CONTROL CARD,229030294.0,NOT RENEWED,N,NaN,DAGMAR,J,NORDLUND,...,None,None,None,N,08/16/2006,None,None,None,None,None
2,365129,COSMO,LICENSED COSMETOLOGIST,11053076.0,NOT RENEWED,N,NaN,RADOJE,NaN,ZELENOVIC,...,None,None,None,N,05/26/2006,None,None,None,None,None
3,595427,COSMO,LICENSED COSMETOLOGIST,11295645.0,ACTIVE,N,NaN,BECKY SUE,L,BURROUGHS,...,None,None,None,N,11/12/2021,None,None,None,None,None
4,653668,COSMO,LICENSED NAIL TECHNICIAN,169006247,NOT RENEWED,N,NaN,BILL G,L,LETNER,...,None,None,None,N,05/30/2006,None,None,None,None,None


In [3]:
licenses[['License Type', 'License Status', 'Expiration Date', 'Effective Date']].dtypes # check the data types

License Type       object
License Status     object
Expiration Date    object
Effective Date     object
dtype: object

### Data preprosessing

#### We need to check whether 'License Type' and 'License Status' columns has null value¶

In [4]:
licenses.loc[licenses['License Type'].isnull() == True]

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason


In [5]:
licenses.loc[licenses['License Status'].isnull() == True]

,_id,License Type,Description,License Number,License Status,Business,Title,First Name,Middle,Last Name,...,Specialty/Qualifier,Controlled Substance Schedule,Delegated Controlled Substance Schedule,Ever Disciplined,LastModifiedDate,Case Number,Action,Discipline Start Date,Discipline End Date,Discipline Reason


 #### We need to convert 'Effective Date', 'Expiration Date' to 'datetime' data type and check whether these two columns has null value

In [6]:
licenses.loc[licenses['Expiration Date'].isna() == True]['Expiration Date'] # check if there has NaN in the column

30      NaN
33      NaN
47      NaN
50      NaN
56      NaN
       ... 
9915    NaN
9933    NaN
9949    NaN
9970    NaN
9997    NaN
Name: Expiration Date, Length: 500, dtype: object

In [7]:
licenses.loc[licenses['Effective Date'].isna() == True]['Effective Date'] # check if there has NaN in the column

33      NaN
47      NaN
50      NaN
56      NaN
68      NaN
       ... 
9940    NaN
9949    NaN
9966    NaN
9972    NaN
9993    NaN
Name: Effective Date, Length: 792, dtype: object

#### Convert ['Effective Date'], ['Expiration Date'] to date time data types

In [8]:
# invalid parsing will be set as NaT
licenses['Effective Date'] = pd.to_datetime(licenses['Effective Date'], errors='coerce', format='%m/%d/%Y')
licenses['Expiration Date'] = pd.to_datetime(licenses['Expiration Date'], errors='coerce', format='%m/%d/%Y')

#### We choose to drop the rows with these NaN values in ['Effective Date', 'Expiration Date']

In [9]:
licenses = licenses.dropna(subset=['Effective Date', 'Expiration Date'])

In [10]:
lic_type_status_data = licenses[['License Type', 'License Status', 'Expiration Date', 'Effective Date']]
heat_map_data = lic_type_status_data.copy()
heat_map_data['Date Diff'] = (licenses['Expiration Date'] - licenses['Effective Date']).dt.days
heat_map_data

,License Type,License Status,Expiration Date,Effective Date,Date Diff
0,DETECTIVE BOARD,NOT RENEWED,2021-09-30,2020-02-03,605
1,DETECTIVE BOARD,NOT RENEWED,2003-12-31,1995-02-07,3249
2,COSMO,NOT RENEWED,1983-09-30,1945-02-28,14093
3,COSMO,ACTIVE,2023-09-30,2021-11-12,687
4,COSMO,NOT RENEWED,2002-10-31,1995-07-12,2668
...,...,...,...,...,...
9994,COSMO,NOT RENEWED,1987-09-30,1983-12-30,1370
9995,DETECTIVE BOARD,NOT RENEWED,1997-05-31,1987-08-04,3588
9996,DETECTIVE BOARD,TERMINATED CARD RETURNED,1990-06-11,1989-05-22,385
9998,COSMO,ACTIVE,2023-09-30,2021-12-17,652


#### Columns are License Type, Rows are the License Status, Values are mean of the days between Effective Date and Expiration Date

In [11]:
heat_map_datatable = pd.pivot_table(heat_map_data, values='Date Diff', index=['License Status'],
                    columns=['License Type'], aggfunc=np.mean, fill_value=0)
heat_map_datatable

License Type,APPRAISAL,ARCHITECT,ATHLETIC TRAINER,ATHLETICS,AUCTIONEER,CEMETERY OVERSIGHT,CLIN PSYCHOLOGIST,COLLECTION AGENCY,COMM ASSOC MGR,COSMO,...,HME AND SERVICES PROV,HOME INSPECTOR,IDPR,INTERIOR DESIGN,LAND SURVEYOR BOARD,LANDSCAPE ARCHITECT,LIMITED LIABILITY CO,MAR AND FAM THERAPIST,MASSAGE LICENSING BD,MEDICAL BOARD
License Status,,,,,,,,,,,,,,,,,,,,,
ACTIVE,0,753.750000,716.0,0.000000,816,0,692.777778,1038.285714,733.421053,687.604255,...,1129.600,661.142857,662.5,652.500000,752.60,773.666667,1123.066667,1085.285714,748.608696,1173.0
CANCELLED,0,0.000000,0.0,0.000000,0,0,0.000000,4115.000000,0.000000,883.100000,...,0.000,0.000000,730.0,772.000000,3652.00,0.000000,0.000000,0.000000,0.000000,0.0
CHANGE OF OWNERSHIP,0,0.000000,0.0,0.000000,0,0,0.000000,1178.000000,0.000000,1034.285714,...,1167.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
CLOSED,0,0.000000,0.0,0.000000,0,0,0.000000,813.500000,0.000000,1222.416667,...,868.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
DECEASED,0,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.000000,5773.400000,...,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
EXPIRED,183,0.000000,0.0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,...,0.000,508.375000,0.0,0.000000,3652.00,0.000000,0.000000,0.000000,0.000000,0.0
INACTIVE,0,5557.500000,0.0,0.000000,0,0,0.000000,0.000000,687.000000,4197.493103,...,0.000,0.000000,0.0,1112.600000,4119.25,3704.000000,0.000000,806.000000,749.916667,8705.5
INOPERATIVE,0,0.000000,0.0,0.000000,0,0,0.000000,1051.000000,0.000000,3653.333333,...,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
NOT RENEWED,0,752.166667,411.5,524.133333,818,1419,774.000000,793.428571,659.000000,2240.072855,...,523.125,693.375000,729.5,1350.785714,3652.00,772.500000,307.333333,1091.166667,730.294118,1912.0


In [12]:
data = heat_map_datatable.values
data.min(), data.max()

(-7.0, 14172.1875)

#### Use data from the 'heat_map_datatable' to create a grid heat map
- x label is the license type; y label is the license status
- the selected cell will display the value which is the days between the effective date and the expiration date

In [13]:
column = heat_map_datatable.columns.values.astype('str') # get x labels
row = heat_map_datatable.index.values.astype('str') # get y labels

value_label = ipywidgets.Label()
def on_selected(change):
    row, col = change['owner'].selected[0]
    v = data[row, col]
    value_label.value = 'Days Mean = '+ str(v)

# Define color, x, y scales
col_sc = bqplot.ColorScale(scheme='Greens')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# Define color, x, y axis
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right', label='DaysMean')
x_ax = bqplot.Axis(scale=x_sc, tick_style={'font-size':10}, tick_rotate=-90, label='License Type', label_offset='50px')
y_ax = bqplot.Axis(scale=y_sc, tick_style={'font-size':10}, orientation='vertical', label='License Status', label_offset='50px')

heat_map = bqplot.GridHeatMap(color=data, row=row, column=column,
                              scales={'color':col_sc, 'row':y_sc, 'column':x_sc}, 
                              interactions={'click':'select'}, 
                              selected_style={'fill':'pink'})

# Define interactions
heat_map.observe(on_selected, 'selected')

fig = bqplot.Figure(marks=[heat_map], axes=[col_ax,x_ax,y_ax], 
                   max_aspect_ratio=100.0)
dashboard = ipywidgets.VBox([value_label, fig])
dashboard

#### To plot a bar plot to show the year and the median of days difference between between the effective date and the expiration date in that year, we need to add a new column called 'Year' which is the year grabbed from the original issue date

In [14]:
licenses['Original Issue Date'] = pd.to_datetime(licenses['Original Issue Date'], errors='coerce', format='%m/%d/%Y')
licenses.loc[licenses['Original Issue Date'].isna() == True]['Original Issue Date'] # check if there has NaN in the column
heat_map_data['Year'] = licenses['Original Issue Date'].dt.year
heat_map_data

,License Type,License Status,Expiration Date,Effective Date,Date Diff,Year
0,DETECTIVE BOARD,NOT RENEWED,2021-09-30,2020-02-03,605,2020
1,DETECTIVE BOARD,NOT RENEWED,2003-12-31,1995-02-07,3249,1995
2,COSMO,NOT RENEWED,1983-09-30,1945-02-28,14093,1945
3,COSMO,ACTIVE,2023-09-30,2021-11-12,687,2011
4,COSMO,NOT RENEWED,2002-10-31,1995-07-12,2668,1995
...,...,...,...,...,...,...
9994,COSMO,NOT RENEWED,1987-09-30,1983-12-30,1370,1983
9995,DETECTIVE BOARD,NOT RENEWED,1997-05-31,1987-08-04,3588,1987
9996,DETECTIVE BOARD,TERMINATED CARD RETURNED,1990-06-11,1989-05-22,385,1989
9998,COSMO,ACTIVE,2023-09-30,2021-12-17,652,1983


#### Use a specific cell in the grid heat map to create a license mask to specify the license type and status

In [15]:
i, j = 4, 10
lc_type = heat_map_datatable.columns[j]
lc_status = heat_map_datatable.index[i]
lc_type, lc_status

('DENTAL', 'DECEASED')

In [16]:
# filter the license status and license type, then create a lincense mark
licenses_mask = ((heat_map_data['License Status'] == lc_status) & (heat_map_data['License Type'] == lc_type) & (heat_map_data['License Status'] == lc_status) & (heat_map_data['License Type'] == lc_type))
licenses_mask

0       False
1       False
2       False
3       False
4       False
        ...  
9994    False
9995    False
9996    False
9998    False
9999    False
Length: 8767, dtype: bool

#### Calculate the median of the difference between between the effective date and the expiration date in that specific cell

In [17]:
year_day_median = heat_map_data[licenses_mask].groupby("Year")['Date Diff'].median()
year_day_median

Year
1930    22698.5
1933    18666.0
1939    17943.0
1943    15045.0
1947    15178.0
1953    15002.0
1955    12099.0
1957    17405.0
1958    13930.0
1961    12289.0
1962    14669.0
1964     7979.0
1990     1697.0
2002     1513.0
Name: Date Diff, dtype: float64

In [18]:
days_diff_median, year_edges = np.histogram(year_day_median.index, 
                                            weights=year_day_median.values,
                                            bins=10)
days_diff_median, year_edges

(array([41364.5, 32988. , 15178. , 58436. , 34937. ,     0. ,     0. ,
            0. ,  1697. ,  1513. ]),
 array([1930. , 1937.2, 1944.4, 1951.6, 1958.8, 1966. , 1973.2, 1980.4,
        1987.6, 1994.8, 2002. ]))

In [19]:
year_centers = (year_edges[:-1] + year_edges[1:]) / 2
year_centers

array([1933.6, 1940.8, 1948. , 1955.2, 1962.4, 1969.6, 1976.8, 1984. ,
       1991.2, 1998.4])

#### Use the data calculated above to plot a bar plot

In [20]:
# Define x, y Linear scales - bar plot
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# Define x, y axis - bar plot
ax_xcl = bqplot.Axis(label='Year', scale=x_scl)
ax_ycl = bqplot.Axis(label='Median of Days Diff Mean', scale=y_scl, orientation='vertical')

# Define marks - bar plot
days_diff_median_hist = bqplot.Bars(x=year_centers, y=days_diff_median, scales={'x':x_scl, 'y':y_scl})

fig_dur = bqplot.Figure(marks=[days_diff_median_hist],axes=[ax_xcl,ax_ycl])

fig_dur

Figure(axes=[Axis(label='Year', scale=LinearScale()), Axis(label='Median of Days Diff Mean', orientation='vert…

#### Link the grid heat map with the bar plot

In [21]:
column = heat_map_datatable.columns.values.astype('str') # get x labels
row = heat_map_datatable.index.values.astype('str') # get y labels

value_label = ipywidgets.Label()
def on_selected(change):
    if len(change['owner'].selected) == 1:
        row, col = change['owner'].selected[0] # make sure only select one cell
        v = data[row, col]
        value_label.value = 'Days Mean = '+ str(v)
        lc_type = heat_map_datatable.columns[col] # select the specific license type
        lc_status = heat_map_datatable.index[row] # select the specific license status
        licenses_mask = ((heat_map_data['License Status'] == lc_status) & (heat_map_data['License Type'] == lc_type) & 
                         (heat_map_data['License Status'] == lc_status) & (heat_map_data['License Type'] == lc_type))
        if len(heat_map_data['Year'][licenses_mask]):
            year_day_median = heat_map_data[licenses_mask].groupby("Year")['Date Diff'].median()
            days_diff_median, year_edges = np.histogram(year_day_median.index, 
                                            weights=year_day_median.values,
                                            bins=10)
            year_centers = (year_edges[:-1] + year_edges[1:]) / 2
            days_diff_median_hist.x = year_centers
            days_diff_median_hist.y = days_diff_median

# Define color, x, y scales - HeatMap
col_sc = bqplot.ColorScale(scheme='RdPu', min=np.nanmin(data), max=np.nanmax(data))
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# Define color, x, y axis - HeatMap
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right', label='DaysMean')
x_ax = bqplot.Axis(scale=x_sc, tick_style={'font-size':10}, tick_rotate=-90, label='License Type', label_offset='50px')
y_ax = bqplot.Axis(scale=y_sc, tick_style={'font-size':10}, orientation='vertical', label='License Status', label_offset='50px')

heat_map = bqplot.GridHeatMap(color=data, row=row, column=column,
                              scales={'color':col_sc, 'row':y_sc, 'column':x_sc}, 
                              interactions={'click':'select'}, 
                              selected_style={'fill':'blue'})

# # Define x, y Linear scales - bar plot
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# Define x, y axis - bar plot
ax_xcl = bqplot.Axis(label='Year', scale=x_scl)
ax_ycl = bqplot.Axis(label='Median of Days Diff Mean', scale=y_scl, orientation='vertical')

# Define marks - bar plot
days_diff_median_hist = bqplot.Bars(x=year_centers, y=days_diff_median, scales={'x':x_scl, 'y':y_scl})

fig_daydiff_median = bqplot.Figure(marks=[days_diff_median_hist],axes=[ax_xcl,ax_ycl])


# Create interactions and link Bar Plot & Heat Map
heat_map.observe(on_selected, 'selected')

fig_heat_map = bqplot.Figure(marks=[heat_map], axes=[col_ax,x_ax,y_ax], 
                   max_aspect_ratio=100.0)

fig_heat_map.layout.min_width='500px'
fig_daydiff_median.layout.min_width='500px'
figures = ipywidgets.HBox([fig_heat_map, fig_daydiff_median])
dashboard = ipywidgets.VBox([value_label, figures])

dashboard

### Summary

#### Transformation & scaling
For the gridheatmap, we set the ordinal scale to remap the license status and license type to a numerical range that can be displayed on the grid heat map. And for the bar plot, we do the linear scale, just because they are two numerical data, which map them into a numerical range.
Before visualizing process, we groupby 'License Type' and 'License Status' as a subset, so that we can visualize a heatmap that the audience can learn each type of license's mainstream status.

#### Null values preprosessing
For the effective date column and expiration date column, they all have NaN values, we first detect how many rows these columns are NaN, then because we need to do a calculation for the difference between the two dates, therefore, if we have NaN which may cause the calculations to get negative value or even NaN value. So we decide to drop these rows which have NaN values in the effective date column and expiration date column.

#### Aesthetics discussion
For the grid heat map, we choose to change the x, and y axis' font size, and rotate the x-axis to avoid severe overlapping (although there is still overlapping), and we also give some offset to the x label and y label to let them give a little bit far away from the axes for a better display. To highlight the interaction function, we specifically choose pink as the 'click' color to distinguish the selected grid from surrounding grids. 